In [1]:
import json
import pandas as pd
import re

# Caminhos
caminho_entrada = "C:\\Users\\willgnnerferreira\\Documents\\Modelos\\Perfil-Conciliação\\Dados\\dados_processos_cejusc_original.json"
caminho_saida = "C:\\Users\\willgnnerferreira\\Documents\\Modelos\\Perfil-Conciliação\\Dados\\dados_processos_cejusc_limpos.json"

# 1. Função para carregar JSON
def carregar_dados_jsonl(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        dados = [json.loads(linha) for linha in f]
    return pd.DataFrame(dados)

# 2. Limpeza do texto jurídico
def limpar_texto_juridico(texto):
    if pd.isnull(texto): return ""

    texto = texto.lower()

    # E-mails
    texto = re.sub(r'\S+@\S+', '', texto)

    # Telefones
    texto = re.sub(r'\(?\d{2}\)?[\s\-]?\d{4,5}[\s\-]?\d{4}', '', texto)

    # CPF/CNPJ/RG
    texto = re.sub(r'\b\d{3}\.?\d{3}\.?\d{3}-?\d{2}\b', '', texto)  # CPF
    texto = re.sub(r'\b\d{2}\.?\d{3}\.?\d{3}/?\d{4}-?\d{2}\b', '', texto)  # CNPJ
    texto = re.sub(r'rg[:nº\s]*\d+', '', texto)

    # CEPs
    texto = re.sub(r'\bcep[:\s]*\d{5}-\d{3}\b', '', texto)

    # Endereços comuns
    texto = re.sub(r'\b(rua|avenida|av|travessa|quadra|qd|lote|lt|nº|número|residencial|bairro|condomínio|casa|bloco|setor|apto|apartamento)\b[\w\s,.\-]*', '', texto)

    # Artigos legais continuam, mas números isolados grandes são removidos
    texto = re.sub(r'\b\d{5,}\b', '', texto)

    # Valores monetários (mantém R$ mas remove duplicatas, centavos verbais)
    texto = re.sub(r'r\$\s?[\d.,]+', '', texto)

    # Palavras repetidas e marcas de formatação
    texto = re.sub(r'(“|”|\"|\*|–|•|■|◆|●|…)', '', texto)
    texto = re.sub(r'[\[\](){}<>]', '', texto)

    # Repetições institucionais comuns
    texto = re.sub(r'eu concilio.*?ganhamos\.', '', texto)
    texto = re.sub(r'poder judici.rio tribunal de justi.a.*?interior', '', texto)

    # Espaços e pontuação
    texto = re.sub(r'\s+', ' ', texto)
    texto = texto.strip()

    return texto

# 3. Aplicar a limpeza e salvar
df = carregar_dados_jsonl(caminho_entrada)
df["inteiro_teor_limpo"] = df["inteiro_teor"].apply(limpar_texto_juridico)

# 4. Exportar para novo JSON
with open(caminho_saida, 'w', encoding='utf-8') as f_out:
    for _, linha in df.iterrows():
        f_out.write(json.dumps(linha.to_dict(), ensure_ascii=False) + '\n')

print("Arquivo limpo salvo com sucesso em:")
print(caminho_saida)

Arquivo limpo salvo com sucesso em:
C:\Users\willgnnerferreira\Documents\Modelos\Perfil-Conciliação\Dados\dados_processos_cejusc_limpos.json


| Etapa                        | Descrição                                                     |
| ---------------------------- | ------------------------------------------------------------- |
| Lowercase                 | Transformação de todo o texto para minúsculas                 |
| Remoção de e-mails        | Eliminação de qualquer padrão tipo `fulano@email.com`         |
| Remoção de telefones      | Ex: `(62) 99999-1234`, `62988881234`, `62 9 9254-5850`        |
| CPF, CNPJ, RG             | Remoção de CPFs, CNPJs e RGs via regex                        |
| Endereços                 | Remoção de termos como “Rua”, “Qd.”, “Lt.”, “CEP”, “nº”, etc. |
| Valores monetários        | Remoção de valores tipo `R$ 1.000,00` mantendo só contexto    |
| Repetições institucionais | Frases como “poder judiciário tribunal...” foram removidas    |
| Caracteres especiais      | Parênteses, aspas, bullets, emojis e marcas desnecessárias    |
| Normalização              | Remoção de múltiplos espaços e pontuações flutuantes          |

In [3]:
import json
import pandas as pd
import re

# Dataset limpo
CAMINHO_ENTRADA = "C:\\Users\\willgnnerferreira\\Documents\\Modelos\\Perfil-Conciliação\\Dados\\dados_processos_cejusc_limpos.json"

# Caminho de saída (com novas features)
CAMINHO_SAIDA = "C:\\Users\\willgnnerferreira\\Documents\\Modelos\\Perfil-Conciliação\\Dados\\dados_processos_cejusc_features.json"

# Funções auxiliares
def contem(texto, padrao):
    return int(bool(re.search(padrao, texto, re.IGNORECASE))) if isinstance(texto, str) else 0

def contar(texto, padrao):
    return len(re.findall(padrao, texto, re.IGNORECASE)) if isinstance(texto, str) else 0

# Função principal para extrair features jurídicas
def extrair_features_juridicas(df, campo="inteiro_teor"):
    df = df.copy()

    df["tem_execucao"] = df[campo].apply(lambda x: contem(x, r"\bexecução\b"))
    df["tem_acordo"] = df[campo].apply(lambda x: contem(x, r"\bacordo\b|\bmediação\b|\bconciliação\b"))
    df["tem_dano_moral"] = df[campo].apply(lambda x: contem(x, r"\bdano[s]? moral"))
    df["tem_protesto"] = df[campo].apply(lambda x: contem(x, r"\bprotest[ao]\b|\bserasa\b|\binadimplente"))
    df["tem_penhora"] = df[campo].apply(lambda x: contem(x, r"\bpenhora\b|\bbacenjud\b|\bbloqueio"))
    df["tem_repeticao_indebito"] = df[campo].apply(lambda x: contem(x, r"\brepeti[cç][aã]o de ind[eé]bito"))
    df["tem_inexistencia_relacao"] = df[campo].apply(lambda x: contem(x, r"\binexist[eê]ncia de rela[cç][aã]o"))
    df["tem_valor_monetario"] = df[campo].apply(lambda x: contem(x, r"r\$ ?[\d\.,]+|\b\d+ ?reais\b"))

    df["n_artigos_lei"] = df[campo].apply(lambda x: contar(x, r"\bart\."))
    df["n_tokens_chave"] = df[campo].apply(lambda x: contar(x, r"\b(execução|acordo|protesto|penhora|dano moral|bacenjud|inadimplemento|extrajudicial|indeniza[cç][aã]o)\b"))

    return df

# 1. Carregar JSON
with open(CAMINHO_ENTRADA, 'r', encoding='utf-8') as f:
    linhas = [json.loads(l) for l in f]

df = pd.DataFrame(linhas)

# 2. Aplicar extração
df = extrair_features_juridicas(df, campo="inteiro_teor")

# 3. Salvar novo JSON
with open(CAMINHO_SAIDA, 'w', encoding='utf-8') as f_out:
    for registro in df.to_dict(orient="records"):
        f_out.write(json.dumps(registro, ensure_ascii=False) + '\n')

print("Novo arquivo salvo com features jurídicas em:")
print(CAMINHO_SAIDA)

Novo arquivo salvo com features jurídicas em:
C:\Users\willgnnerferreira\Documents\Modelos\Perfil-Conciliação\Dados\dados_processos_cejusc_features.json
